#### Open the project and print some information (10 pnts)

In [9]:
import arcpy
import os
pathname = r"C:\Users\adeni\Documents\grad\lsu\syllabus\Fall2024\GEOG_4057\assignment\lab8\ex8"
aprx = arcpy.mp.ArcGISProject(os.path.join(pathname,"Austin.aprx"))
print(aprx.defaultGeodatabase)

C:\Users\adeni\Documents\grad\lsu\syllabus\Fall2024\GEOG_4057\assignment\lab8\ex8\Austin_Data.gdb


#### Save a copy of the project (10 pnts)

In [3]:
aprx.saveACopy(os.path.join(pathname,'Austin_Copy.aprx'))

#### Examine the new project file (10 pnts)

- Everything is the same as the original project.

- The size of the original project file is 60 kb and the size of the copied project file is 48 kb.

- Why?
- Answer:
 ArcPy often only stores the essential data connections and processing steps, while the original project file might contain additional information like symbology, layouts, and potentially unnecessary metadata, resulting in a larger file size.

### Work with maps

#### Use the following code to print a list of maps in the project (10 pnts)

In [1]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
Region
Foot_US


- The `del` aprx line in this ArcPy code doesn't delete the actual project file - it just removes the Python variable/object reference to the ArcGIS Project. 
- This is a cleanup step that's considered good practice when working with ArcPy.

#### Use the following code to change the name of a map (10 pnts)

In [1]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Region")[0]
m.name = "County"
del aprx

#### Use the following code to list the layers in a map (10 pnts)

In [3]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)
del aprx

Map: Downtown
trees
parks
base
Topographic
Map: County
facilities
hospitals
parks
Topographic


- Additionally, use the following code to print if a layer is a basemap or a feature layer

In [4]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
lyrs = m.listLayers()
for lyr in lyrs:
    if lyr.isBasemapLayer:
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx

trees is a feature layer
parks is a feature layer
base is a feature layer
Topographic is a basemap layer


#### Change the basemap (5 pnts)

- Use the following code to change the basemap of Downtown

In [6]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")

### Work with layers

#### Modify layer symbology (5 pnts)

In [6]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [100, 175, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

In this code, `green` is a Python dictionary with a single key "RGB" whose value is a list of 4 numbers representing RGBA color values:

- First 3 numbers [100, 175, 0] represent Red, Green, Blue values (0-255 scale)
- The last number (100) represents Alpha/opacity (0-255 scale, where 255 is fully opaque)

##### Modifying the color to brown

The numbers to create a brown color would be {"RGB": [139, 69, 19, 100] } this would create a medium brown color.

In [3]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
brown = {"RGB": [139, 69, 19, 100]}  # A medium brown color
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = brown
    lyr.symbology = sym

### Work with a layout

#### Add a layout to the project ((5 pnts))

In [4]:
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")

- What is the size of the layout in inches?\
Answer - Width: 11 inch, Height: 8.5 inch
- What is the size of the map frame?\
Answer - Width: 10 inch, Height: 7.5 inch
- Comparing the layout and map frames you created manually, what are the advantages of creating it in Python?\
Answer
       -  Multiple layouts can be created quickly.
       -  Dimensions for the layout can be specified by the user.
       -  Can loop through different maps or data sources.
       -  Particularly valuable for repetitive tasks.

#### Add layout elements (5 pnts)

- Run the following code to create a north arrow and a scale bar

In [5]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')

- Run the follow code to create a legend

In [7]:
legSi = aprx.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3' )[0]
leg = lyt.createMapSurroundElement(arcpy.Point(1,7), 'LEGEND', mf, legSi, 'New Legend Element')
leg.elementWidth = 3
leg.elementHeight = 3
leg.fittingStrategy = 'AdjustFontSize'
leg.columnCount = 1
leg.title = 'Downtown'

#### Export the layout to a pdf (10 pnts)

- Run the following code to export the layout to a pdf file

In [10]:
pathname = r"C:\Users\adeni\Documents\grad\lsu\syllabus\Fall2024\GEOG_4057\assignment\lab8\ex8"
lyt.exportToPDF(os.path.join(pathname, 'downtown.pdf'))

'C:\\Users\\adeni\\Documents\\grad\\lsu\\syllabus\\Fall2024\\GEOG_4057\\assignment\\lab8\\ex8\\downtown.pdf'